In [ ]:
import os
import sys
import time

sys.path.append('../..')
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import matplotlib.pyplot as plt
import mdtraj as md
import numpy as np

from ase.io import Trajectory
from Constants import EV_TO_KJ, H_TO_KJ, BOHR_TO_ANGSTROM
from Utilities import show_results
from scipy.constants import Boltzmann, Avogadro
from scipy.stats import pearsonr
import openmm.unit as u

In [ ]:
SYSTEMS_DATA = np.load('systems/systems_gromos.npy', allow_pickle=True).item()


RT = lambda T: (Boltzmann * Avogadro) * T / 1000
RMSE = lambda x, y, n=1: np.round(np.sqrt(np.mean(np.square(np.subtract(x, y)))), n)
MAE =  lambda x, y, n=1: np.round(np.mean(np.abs(np.subtract(x, y))), n) 
MARE =  lambda x, y, n=1: np.round(np.mean(np.abs(np.subtract(x, y)) / x), n) 
ME =  lambda x, y, n=1: np.round(np.mean(np.subtract(x, y)), n)
STD =  lambda x, y, n=1: np.round(np.std(np.subtract(x, y)), n)
MAE =  lambda x, y, n=1: np.round(np.mean(np.abs(np.subtract(x, y))), n)

# Un-tested implementation for alpha from mdtraj
gas_constant = 8.3144621 * u.joule / u.kelvin / u.mole
def thermal_expansion_alpha_P(traj, temperature, energies):
    # Had some issues finding a useful unit test, so disabled this code for now.
    # Feel free to file a pull request with a working unit test :)
    temperature = temperature * u.kelvin
    mean_volume = traj.unitcell_volumes.mean()
    alpha = np.cov(traj.unitcell_volumes, energies)[0, 1]  # <HV> - <H><V> = cov(H, V)
    alpha /= mean_volume
    alpha *= u.kilojoules_per_mole
    alpha /= (gas_constant * temperature ** 2)
    return alpha * u.kelvin

def average_data(data, start, end):
    data = np.array([x[start:end] for x in data])
    mean = np.mean(data, axis=1)
    mean, std = np.mean(mean), np.std(mean)
    return mean, std

In [ ]:
FOLDER_PATH = 'data_ase/'
N_REPLICA = 3

In [ ]:
for SYSTEM_NAME in SYSTEMS_DATA:
    for REPLICA in range(N_REPLICA):
        try:            
            NAME_TRAJFILE = f'{FOLDER_PATH}{SYSTEM_NAME}/{SYSTEM_NAME}_{REPLICA}.traj'
            traj_ase = Trajectory(NAME_TRAJFILE)
            NAME_PDBFILE = NAME_TRAJFILE[:-4] + 'pdb'
            cells = []
            try:
                os.remove(NAME_PDBFILE)
            except:
                pass
            for atoms in traj_ase:
                atoms.write(NAME_PDBFILE, append=True)
                cells.append(atoms.get_cell().array)
            cells = np.array(cells)

            trajectory = md.load(NAME_PDBFILE)
            trajectory.unitcell_vectors = 0.1 * cells
            trajectory.save_dcd(NAME_TRAJFILE[:-4] + 'dcd')
            trajectory[0].save_pdb(NAME_TRAJFILE[:-5] + '_topo.pdb')
        except:
            print(SYSTEM_NAME, REPLICA)

In [ ]:
for SYSTEM_NAME in SYSTEMS_DATA:
    for REPLICA in range(N_REPLICA):
        for re in range(2, 3):
            try:            
                NAME_TRAJFILE = f'{FOLDER_PATH}{SYSTEM_NAME}/{SYSTEM_NAME}_{REPLICA}_re{re}.traj'
                traj_ase = Trajectory(NAME_TRAJFILE)
                NAME_PDBFILE = NAME_TRAJFILE[:-4] + 'pdb'
                cells = []
                try:
                    os.remove(NAME_PDBFILE)
                except:
                    pass
                for atoms in traj_ase:
                    atoms.write(NAME_PDBFILE, append=True)
                    cells.append(atoms.get_cell().array)
                cells = np.array(cells)
        
                trajectory = md.load(NAME_PDBFILE)
                trajectory.unitcell_vectors = 0.1 * cells
                trajectory.save_dcd(NAME_TRAJFILE[:-4] + 'dcd')
                trajectory[0].save_pdb(NAME_TRAJFILE[:-5] + '_topo.pdb')
            except:
                print(SYSTEM_NAME, REPLICA)

In [ ]:
TIME_OFFSET = 0
DATA = {}

In [ ]:
for SYSTEM_NAME in SYSTEMS_DATA:
    DATA[SYSTEM_NAME] = {}
    DATA[SYSTEM_NAME]['density'] = []
    DATA[SYSTEM_NAME]['kappa'] = []
    DATA[SYSTEM_NAME]['alpha'] = []
    DATA[SYSTEM_NAME]['V_pot_monomer'] = []
    DATA[SYSTEM_NAME]['V_pot_liquid'] = []
    DATA[SYSTEM_NAME]['T_monomer'] = []
    DATA[SYSTEM_NAME]['T_system'] = []
    T = SYSTEMS_DATA[SYSTEM_NAME]['T']
    for REPLICA in range(N_REPLICA):        
        NAME_LOGFILE = f'{FOLDER_PATH}{SYSTEM_NAME}/{SYSTEM_NAME}_{REPLICA}.log'
        NAME_LOFGILE_MONOMER = f'data_ase_monomer/{SYSTEM_NAME}/{SYSTEM_NAME}_MONOMER{REPLICA}.log'
        NAME_TRAJFILE_MONOMER = f'data_ase_monomer/{SYSTEM_NAME}/{SYSTEM_NAME}_MONOMER{REPLICA}.traj'
        V_pot_system, T_system = np.loadtxt(NAME_LOGFILE, skiprows=1, usecols=[1, 4], unpack=True)        
        V_pot_monomer, T_monomer = np.loadtxt(NAME_LOFGILE_MONOMER, skiprows=1, usecols=[1, 4], unpack=True)
        for re in [1, 2]:
            NAME_LOGFILE = f'{FOLDER_PATH}{SYSTEM_NAME}/{SYSTEM_NAME}_{REPLICA}_re{re}.log'
            V_pot_system_, T_system_ = np.loadtxt(NAME_LOGFILE, skiprows=1, usecols=[1, 4], unpack=True)
            V_pot_system = np.hstack((V_pot_system, V_pot_system_))
            T_system = np.hstack((T_system, T_system_))
        V_pot_system *= EV_TO_KJ
        V_pot_monomer *= EV_TO_KJ
        FILE_NAME = f'{FOLDER_PATH}{SYSTEM_NAME}/{SYSTEM_NAME}_{REPLICA}'
        traj = md.load([f'{FILE_NAME}.dcd', f'{FILE_NAME}_re1.dcd', f'{FILE_NAME}_re2.dcd'], top=f'{FILE_NAME}_topo.pdb')    
        n_molecules = traj.n_atoms // len(Trajectory(NAME_TRAJFILE_MONOMER)[0])
        DATA[SYSTEM_NAME]['density'].append(md.density(traj))
        DATA[SYSTEM_NAME]['kappa'].append(md.isothermal_compressability_kappa_T(traj[TIME_OFFSET:], T) / (10 ** -5))
        index = min(len(traj), len(V_pot_system))
        DATA[SYSTEM_NAME]['alpha'].append(thermal_expansion_alpha_P(traj[:index], T, V_pot_system[:index]) / 10**-4)
        DATA[SYSTEM_NAME]['V_pot_liquid'].append(V_pot_system / n_molecules)
        DATA[SYSTEM_NAME]['V_pot_monomer'].append(V_pot_monomer)
        DATA[SYSTEM_NAME]['T_system'].append(T_system)
        DATA[SYSTEM_NAME]['T_monomer'].append(T_monomer)  

In [ ]:
names = []
T_start, T_end = 2000, 4000
exp_density, sim_density, std_density = [], [], []
exp_temp, sim_temp, std_temp = [], [], []
exp_kappa, sim_kappa, std_kappa = [], [], []
exp_alpha, sim_alpha, std_alpha = [], [], []
exp_H_vap, sim_H_vap, std_H_vap = [], [], []
for SYSTEM_NAME in SYSTEMS_DATA:
    if np.amax([x.size for x in DATA[SYSTEM_NAME]['density']]) >= T_end:
        mean_density_ = np.array([np.mean(x[T_start:T_end]) for x in DATA[SYSTEM_NAME]['density'] if x[T_start:T_end].any()])
        std_density_ = np.std(mean_density_)
        mean_temp_ = np.mean(np.array([np.mean(x) for x in DATA[SYSTEM_NAME]['T_system'] if x[T_start:T_end].any()]))
        std_temp_ = np.std(mean_temp_)
        mean_kappa_, std_kappa_ = np.mean(DATA[SYSTEM_NAME]['kappa']), np.std(DATA[SYSTEM_NAME]['kappa'])
        mean_alpha_, std_alpha_ = np.mean(DATA[SYSTEM_NAME]['alpha']), np.std(DATA[SYSTEM_NAME]['alpha'])
        V_pot_monomer = np.array([np.mean(x) for x in DATA[SYSTEM_NAME]['V_pot_monomer']]) # [T_start:T_end]
        V_pot_liquid = np.array([np.mean(x[T_start:T_end]) for x in DATA[SYSTEM_NAME]['V_pot_liquid'] if x[T_start:T_end].any()])
        Hvaps = V_pot_monomer[:V_pot_liquid.shape[0]] + RT(T) - V_pot_liquid
        V_pot_mono, V_pot_system = [], []
        exp_H_vap.append(SYSTEMS_DATA[SYSTEM_NAME]['Hvap'])
        sim_H_vap.append(np.mean(Hvaps))
        std_H_vap.append(np.std(Hvaps))
        exp_density.append(SYSTEMS_DATA[SYSTEM_NAME]['density'] * 1e3)
        sim_density.append(np.mean(mean_density_))
        std_density.append(std_density_)       
        exp_temp.append(SYSTEMS_DATA[SYSTEM_NAME]['T'])
        sim_temp.append(mean_temp_)
        std_temp.append(std_temp_)
        if SYSTEMS_DATA[SYSTEM_NAME]['kappa'] is not None:
            exp_kappa.append(SYSTEMS_DATA[SYSTEM_NAME]['kappa'])
            sim_kappa.append(mean_kappa_)
            std_kappa.append(std_kappa_)
        if SYSTEMS_DATA[SYSTEM_NAME]['alpha'] is not None:
            exp_alpha.append(SYSTEMS_DATA[SYSTEM_NAME]['alpha'])
            sim_alpha.append(mean_alpha_)
            std_alpha.append(std_alpha_)
        names.append(SYSTEM_NAME)
        if False:
            for x in range(N_REPLICA):
                plt.plot(DATA[SYSTEM_NAME]['density'][x])
            plt.hlines(SYSTEMS_DATA[SYSTEM_NAME]['density'] * 1e3, T_start, T_end, color='black')
            plt.hlines(mean_density_, T_start, T_end, color='red')
            ax = plt.gca()
            ax.set_title(SYSTEM_NAME)
            plt.show()
        if False:
            for x in range(N_REPLICA):
                plt.plot(DATA[SYSTEM_NAME]['T_system'][x])
            plt.hlines(SYSTEMS_DATA[SYSTEM_NAME]['T'], T_start, T_end, color='black')
            plt.hlines(mean_temp_, T_start, T_end, color='red')
            ax = plt.gca()
            ax.set_title(SYSTEM_NAME)
            ax.set_ylim(250, 350)
            plt.show()
            print(mean_temp_)
    else:
        print(SYSTEM_NAME)
        print([x.size for x in DATA[SYSTEM_NAME]['density']])

In [ ]:
exp_H_vap, sim_H_vap, std_H_vap = np.array(exp_H_vap), np.array(sim_H_vap), np.array(std_H_vap)
exp_density, sim_density, std_density = np.array(exp_density), np.array(sim_density), np.array(std_density)
exp_kappa, sim_kappa, std_kappa = np.array(exp_kappa), np.array(sim_kappa), np.array(std_kappa)
exp_alpha, sim_alpha, std_alpha = np.array(exp_alpha), np.array(sim_alpha), np.array(std_alpha)
exp_temp, sim_temp = np.array(exp_temp), np.array(sim_temp)

In [ ]:
def print_out_results(target=None, prediction=None, stds=None, property_name='', unit='', n_digits=1):
    std = np.round(np.mean(stds), n_digits)
    mae = MAE(target, prediction, n_digits)
    mare = MARE(target, prediction, n_digits)
    rmse = RMSE(target, prediction, n_digits)
    me = ME(target, prediction, n_digits)
    max_abs = np.round(np.amax(np.abs(prediction - target)), n_digits)
    r = np.round(pearsonr(exp_H_vap, sim_H_vap).statistic, 2)
    print(f'{property_name} [{unit}]& {rmse:{4}.{n_digits}f} $\pm$ {std:{4}.{n_digits}f}\\')
    print(f'{property_name} [{unit}]& {mae:{4}.{n_digits}f} $\pm$ {std:{4}.{n_digits}f} & {me:{4}.{n_digits}f} & {rmse:{4}.{n_digits}f} & {mare:{4}.{n_digits}f} & {max_abs:{4}.{n_digits}f} & {r:{4}.{2}f}\\')

In [ ]:
print_out_results(target=exp_H_vap, prediction=sim_H_vap, stds=std_H_vap, property_name="H$_{\\text{vap}}", unit='kJ/mol', n_digits=1)
print_out_results(target=exp_density, prediction=sim_density, stds=std_density, property_name="Density", unit='kg\\cdot m$^{-3}$', n_digits=1)

In [ ]:
np.mean(std_H_vap), np.mean(std_kappa), np.mean(std_alpha)

In [ ]:
show_results(exp_temp, sim_temp, names=names, dataset_name='Temperature', show_plot=True, show_mae=False, unit='K')

In [ ]:
show_results(exp_H_vap, sim_H_vap, names=None, dataset_name='Heat of Vaporization', show_plot=True, show_mae=False)

In [ ]:
show_results(exp_density, sim_density, names=None, dataset_name='Density', show_plot=True, show_mae=False)

In [ ]:
subfig, axs = plt.subplots(1, 2, figsize=(9, 4), dpi=300)
axs[0].scatter(exp_density, sim_density, s=35, color='#3E9BBD', edgecolors='black', linewidths=.1)
axs[0].set_ylim(650, 1150)
axs[0].set_xlim(650, 1150)
axs[0].spines['top'].set_visible(False)
axs[0].spines['right'].set_visible(False)
axs[0].spines['bottom'].set_visible(False)
axs[0].spines['left'].set_visible(False)
xs = np.linspace(650, 1150)
axs[0].plot(xs, xs, color='black', linewidth=1, zorder=-1)
axs[0].plot(xs, xs + 50, linewidth=.4, color='grey', zorder=-1)
axs[0].plot(xs, xs - 50, linewidth=.4, color='grey', zorder=-1)
axs[0].set_xlabel('Experimental Density [kg$\cdot$m$^{-3}$]')
axs[0].set_ylabel('ANA2B$^1$ Density [kg$\cdot$m$^{-3}$]')
axs[0].set_title('Density')

axs[1].scatter(exp_H_vap, sim_H_vap, s=35, color='#3E9BBD', edgecolors='black', linewidths=.1)
axs[1].set_ylim(15, 80)
axs[1].set_xlim(15, 80)
axs[1].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)
axs[1].spines['bottom'].set_visible(False)
axs[1].spines['left'].set_visible(False)
xs = np.linspace(15, 80)
axs[1].plot(xs, xs, color='black', linewidth=1, zorder=-1)
axs[1].plot(xs, xs + 4.184, linewidth=.4, color='grey', zorder=-1)
axs[1].plot(xs, xs - 4.184, linewidth=.4, color='grey', zorder=-1)
axs[1].set_xlabel('Experimental H$_{Vap}$ [kJ/mol]')
axs[1].set_ylabel('ANA2B$^1$ H$_{Vap}$ [kJ/mol]')
axs[1].set_title('Heat of Vaporization')
plt.savefig('figures/results_MD.pdf', bbox_inches='tight')

In [ ]:
show_results(exp_kappa, sim_kappa, dataset_name='Kappa', show_plot=True, show_mae=False)

In [ ]:
show_results(exp_alpha, sim_alpha, dataset_name='Alpha', show_plot=True, show_mae=False)

In [ ]:
tab_string = ''
N_DIGITS = 2
for name, hvap_exp, hvap_pred, hvap_std, rho_exp, rho_pred, rho_std, t_exp, t_sim  in zip(names, exp_H_vap, sim_H_vap, std_H_vap, exp_density, sim_density, std_density, exp_temp, sim_temp):
    hvap_exp = np.round(hvap_exp, N_DIGITS)
    hvap_pred = np.round(hvap_pred, N_DIGITS)
    hvap_std = np.round(hvap_std, N_DIGITS)
    rho_exp = np.round(rho_exp, N_DIGITS)
    rho_pred = np.round(rho_pred, N_DIGITS)
    rho_std = np.round(rho_std, N_DIGITS)
    t_exp = np.round(t_exp, N_DIGITS)
    t_sim = np.round(t_sim, N_DIGITS)
    tab_string += f'{name} & {hvap_exp:{4}.{N_DIGITS}f} & {hvap_pred:{4}.{N_DIGITS}f}$\pm${hvap_std:{4}.{N_DIGITS}f} & {rho_exp:{4}.{N_DIGITS}f}  & {rho_pred:{4}.{N_DIGITS}f}$\pm${rho_std:{4}.{N_DIGITS}f} & {t_exp:{4}.{N_DIGITS}f}  & {t_sim:{4}.{N_DIGITS}f}\\\\\n'
print(tab_string)

In [ ]:
np.where(np.array(names) == 'EDAN')

In [ ]:

sim_H_vap
    

In [ ]:
exp_H_vap, sim_H_vap, std_H_vap = np.array(exp_H_vap), np.array(sim_H_vap), np.array(std_H_vap)
exp_density, sim_density, std_density = np.array(exp_density), np.array(sim_density), np.array(std_density)
exp_kappa, sim_kappa, std_kappa = np.array(exp_kappa), np.array(sim_kappa), np.array(std_kappa)
exp_alpha, sim_alpha, std_alpha = np.array(exp_alpha), np.array(sim_alpha), np.array(std_alpha)
exp_temp, sim_temp = np.array(exp_temp), np.array(sim_temp)

In [ ]:
SYSTEM_NAME = 'EDAN'
mean_density_ = np.array([np.mean(x[T_start:T_end]) for x in DATA[SYSTEM_NAME]['density'] if x[T_start:T_end].any()])
std_density_ = np.std(mean_density_)
mean_temp_ = np.mean(np.array([np.mean(x) for x in DATA[SYSTEM_NAME]['T_system'] if x[T_start:T_end].any()]))
std_temp_ = np.std(mean_temp_)
mean_kappa_, std_kappa_ = np.mean(DATA[SYSTEM_NAME]['kappa']), np.std(DATA[SYSTEM_NAME]['kappa'])
mean_alpha_, std_alpha_ = np.mean(DATA[SYSTEM_NAME]['alpha']), np.std(DATA[SYSTEM_NAME]['alpha'])
V_pot_monomer = np.array([np.mean(x) for x in DATA[SYSTEM_NAME]['V_pot_monomer']]) # [T_start:T_end]
V_pot_liquid = np.array([np.mean(x[T_start:T_end]) for x in DATA[SYSTEM_NAME]['V_pot_liquid'] if x[T_start:T_end].any()])
Hvaps = V_pot_monomer[:V_pot_liquid.shape[0]] + RT(T) - V_pot_liquid
V_pot_mono, V_pot_system = [], []


In [ ]:
Hvaps

In [ ]:
mean_density_

In [ ]:
DATA[SYSTEM_NAME]['density'][0][T_start:T_end]

In [ ]:
Hvaps = V_pot_monomer[1:] + RT(T) - V_pot_liquid[1:]
np.mean(Hvaps), np.std(Hvaps)

In [ ]:
V_pot_monomer, V_pot_liquid

In [ ]:
DATA['EDAN']['density']

In [ ]:
DATA['EDAN']['V_pot_liquid']

In [ ]:
DATA['EDAN']['V_pot_monomer']

In [ ]:
exp_H_vap.append(SYSTEMS_DATA[SYSTEM_NAME]['Hvap'])
sim_H_vap.append(np.mean(Hvaps))
std_H_vap.append(np.std(Hvaps))
exp_density.append(SYSTEMS_DATA[SYSTEM_NAME]['density'] * 1e3)
sim_density.append(np.mean(mean_density_))
std_density.append(std_density_)       
exp_temp.append(SYSTEMS_DATA[SYSTEM_NAME]['T'])
sim_temp.append(mean_temp_)
std_temp.append(std_temp_)